Title:

Introduction

#Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal

#Clearly state the question you will try to answer with your project

#Identify and describe the dataset that will be used to answer the question

#significance of this project


Terminology



Preliminary exploratory data analysis below

In [ ]:
library(tidyverse)
readxl

In [4]:
#reading the data
# <- read_csv("data/")

In [ ]:
#cleaning the data

In [5]:
#summarize the data (eg mean, standard deviation, number of observations)

In [ ]:
#correlation matrix

In [ ]:
#visualising the data across two variables 

In [6]:
#visualising the data across another two variables 

Methods


#Explain how you will conduct either your data analysis and which variables/columns you will use

#Describe at least one way that you will visualize the results


Expected outcomes and significance

#What do you expect to find?

#What impact could such findings have?

#What future questions could this lead to?
